# Transportation Optimization

This analysis uses linear optimization to find an optimal delivery route for a delivery company.

Description:

A delivery company that owns six vehicles ($V_0,\dots,V_5$) is providing a delivery service to 16 locations. In total, the vehicles travel 17 locations ($L_0,\dots,L_{16}$) including the headquarter ($L_0$). 

![](https://drive.google.com/uc?id=1CjVp2nwKAjqy0nsJ9Kj57_Uju5ZmPfQl)

The company wants to make a vehicle routing plan to reduce the transportation cost where the total transportation cost is proportional to the total travel distances. The travel times (in minutes) between any two locations are given in the following table. 

| <i></i> |L0 |  L1|  L2|  L3|  L4|  L5|  L6|  L7|  L8|  L9| L10| L11| L12| L13| L14| L15| L16|
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
|**L0**        |0 | 36| 48| 17| 57| 41| 47| 51| 31| 24| 46| 68| 44| 32| 48| 37| 20|
|**L1**        |36|  0| 51| 30| 25| 20| 35| 47|  5| 13| 19| 42| 11| 27| 43| 41| 37|
|**L2**        |48| 51|  0| 32| 47| 32| 21|  9| 49| 51| 39| 45| 62| 23| 10| 11| 28|
|**L3**        |17| 30| 32|  0| 43| 25| 30| 34| 26| 22| 32| 52| 40| 15| 30| 21| 7 |
|**L4**        |57| 25| 47| 43|  0| 18| 26| 39| 30| 38| 10| 17| 31| 31| 37| 42| 48|
|**L5**        |41| 20| 32| 25| 18|  0| 14| 27| 22| 27|  8| 27| 31| 13| 24| 25| 30|
|**L6**        |47| 35| 21| 30| 26| 14|  0| 14| 35| 40| 19| 25| 45| 14| 11| 18| 31|
|**L7**        |51| 47|  9| 34| 39| 27| 14|  0| 47| 50| 33| 36| 58| 22|  4| 14| 32|
|**L8**        |31|  5| 49| 26| 30| 22| 35| 47|  0|  8| 22| 45| 14| 26| 43| 39| 33|
|**L9**        |24| 13| 51| 22| 38| 27| 40| 50|  8|  0| 29| 53| 20| 27| 46| 40| 29|
|**L10**       |46| 19| 39| 32| 10|  8| 19| 33| 22| 29|  0| 23| 27| 21| 30| 33| 37|
|**L11**       |68| 42| 45| 52| 17| 27| 25| 36| 45| 53| 23|  0| 48| 37| 35| 44| 55|
|**L12**       |44| 11| 62| 40| 31| 31| 45| 58| 14| 20| 27| 48|  0| 38| 54| 53| 47|
|**L13**       |32| 27| 23| 15| 31| 13| 14| 22| 26| 27| 21| 37| 38|  0| 18| 14| 17|
|**L14**       |48| 43| 10| 30| 37| 24| 11| 4|  43| 46| 30| 35| 54| 18|  0| 12| 29|
|**L15**       |37| 41| 11| 21| 42| 25| 18| 14| 39| 40| 33| 44| 53| 14| 12|  0| 17|
|**L16**       |20| 37| 28|  7| 48| 30| 31| 32| 33| 29| 37| 55| 47| 17| 29| 17|  0|

The objective to find an optimal routing schedule that minimizes the total travel time subject to the following constraints. *The returning trips to the headquarter are not considered as parts of routes.*

**Constraint 1:** Each vehicle can travel no more than 100 minutes. *The returning time from the last location to the headquarter is not counted toward the 100 minutes.*

**Constraint 2:** Each location except for the headquarter must be visited by exactly one vehicle.

**Constraint 3:** All the used vehicles must depart from the headquarter. 

Without loss of generality, we assume that the vehicle $i$ visits more or equal locations than vehicle $j$ for all $i\le j$.

In [1]:
# Pip-install the gurobipy package
%pip install gurobipy

In [2]:
import numpy as np

# Define n, K, and c
# number of available vehicles
K = 6

# number of travelling locations
n = 17

# maximum allowed travel time per vehicle (in minutes)
T = 100

# travel time between all locations
c = np.array(
[[0,  36, 48, 17, 57, 41, 47, 51, 31, 24, 46, 68, 44, 32, 48, 37, 20],
 [36, 0,  51, 30, 25, 20, 35, 47, 5,  13, 19, 42, 11, 27, 43, 41, 37],
 [48, 51, 0,  32, 47, 32, 21, 9,  49, 51, 39, 45, 62, 23, 10, 11, 28],
 [17, 30, 32, 0,  43, 25, 30, 34, 26, 22, 32, 52, 40, 15, 30, 21, 7 ],
 [57, 25, 47, 43, 0,  18, 26, 39, 30, 38, 10, 17, 31, 31, 37, 42, 48],
 [41, 20, 32, 25, 18, 0,  14, 27, 22, 27, 8,  27, 31, 13, 24, 25, 30],
 [47, 35, 21, 30, 26, 14, 0,  14, 35, 40, 19, 25, 45, 14, 11, 18, 31],
 [51, 47, 9,  34, 39, 27, 14, 0,  47, 50, 33, 36, 58, 22, 4,  14, 32],
 [31, 5,  49, 26, 30, 22, 35, 47, 0,  8,  22, 45, 14, 26, 43, 39, 33],
 [24, 13, 51, 22, 38, 27, 40, 50, 8,  0,  29, 53, 20, 27, 46, 40, 29],
 [46, 19, 39, 32, 10, 8,  19, 33, 22, 29, 0,  23, 27, 21, 30, 33, 37],
 [68, 42, 45, 52, 17, 27, 25, 36, 45, 53, 23, 0,  48, 37, 35, 44, 55],
 [44, 11, 62, 40, 31, 31, 45, 58, 14, 20, 27, 48, 0,  38, 54, 53, 47],
 [32, 27, 23, 15, 31, 13, 14, 22, 26, 27, 21, 37, 38, 0,  18, 14, 17],
 [48, 43, 10, 30, 37, 24, 11, 4,  43, 46, 30, 35, 54, 18, 0,  12, 29],
 [37, 41, 11, 21, 42, 25, 18, 14, 39, 40, 33, 44, 53, 14, 12, 0,  17],
 [20, 37, 28, 7,  48, 30, 31, 32, 33, 29, 37, 55, 47, 17, 29, 17, 0]]
 )

## Initial Analysis

The first step is to understand the route time for some example routes, as well as understand which sample routes are feasible or not feasible, based on the constraints

In [3]:
# define a function to calculate route time
def route_time(route):
  indexer = 0

  travel_times = []

  # iterate through locations except the last one
  for loc in route[:-1]:
    start_loc = route[indexer]
    end_loc = route[indexer + 1]

    # exclude travel time back to headquarters
    if(end_loc) == 0:
      pass
    else:
      # get travel times from table
      time = c[start_loc][end_loc]
      travel_times.append(time)

    indexer += 1

  return sum(travel_times)

In [4]:
# print a sample route
print(route_time([0, 6, 2, 4, 9, 8]))

161


## Route Samples

The analysis below determines whether some sample routes are feasible, and what the routing time is for the sample route.

In [5]:
# Route 1 visits all locations in chronological order
print('Route 1 travel time:', route_time([i for i in range(17)]))

# route 1 is not feasible since vehicle 0 travels for more than 100 minutes

Route 1 travel time: 448


In [6]:
# Route 2 uses Vehicle 0 on 2 routes, and two other vehicles (V1 and V2)
print('V0 first route time', route_time([0, 1]))
print('V0 second route time', route_time([2, 3, 4, 5]))
print('V1 route time', route_time([0,7,8,9,10,11]))
print('V2 route time', route_time([12,13,14,15,16]))

# Route 2 is infeasible because
# (1) V2 does not depart from the headquarters
# (2) V1 route time is more than 100 minutes
# (3) Location #6 is not visited in the routing

V0 first route time 36
V0 second route time 93
V1 route time 158
V2 route time 85


In [7]:
# Route 3 uses 3 vehicles (V0, V1, and V2)
print('V0 route time', route_time([0,1,2,3,4,5]))
print('V1 route time', route_time([0,7,8,9,10,11]))
print('V2 route time', route_time([0,12,13,14,15,16]))

# this routing is infeasible because all 3 vehicles used travel for more than 100 minutes

V0 route time 180
V1 route time 158
V2 route time 129


In [8]:
# Route 4 uses all 6 vehicles
print('V0 route time: ', route_time([0,1,2]))
print('V1 route time: ', route_time([0,3,2,4]))
print('V2 route time: ', route_time([0,7,5,6]))
print('V3 route time: ', route_time([0,9,8,10,11]))
print('V4 route time: ', route_time([0,13,12]))
print('V5 route time: ', route_time([0,16,14,15]))

# Routing 4 is infeasible because: 
#   location 2 is visited by more than 1 vehicle

V0 route time:  87
V1 route time:  96
V2 route time:  92
V3 route time:  77
V4 route time:  70
V5 route time:  61


In [9]:
# Route 5 uses also uses all 6 vehicles
v0 = route_time([0,1,2])
v1 = route_time([0,3,4])
v2 = route_time([0,7,5,6])
v3 = route_time([0,9,8,10,11])
v4 = route_time([0,13,12])
v5 = route_time([0,16,14,15])

print('V0 route time: ', v0)
print('V1 route time: ', v1)
print('V2 route time: ', v2)
print('V3 route time: ', v3)
print('V4 route time: ', v4)
print('V5 route time: ', v5)

print('\nTotal routing time: ', v0 + v1 + v2 + v3 + v4 + v5)

# Routing 5 is feasible because:
#     all vehicles depart from location 0
#     all locations are visited only once
#     all vehicles travel for less than 100 minutes total

V0 route time:  87
V1 route time:  60
V2 route time:  92
V3 route time:  77
V4 route time:  70
V5 route time:  61

Total routing time:  447


In [10]:
# create a sample feasible routing plan, that is more optimized than Route 5
vehicle_0 = route_time([0,1,2])
vehicle_1 = route_time([0,3,4])
vehicle_2 = route_time([0,5,6,7])
vehicle_3 = route_time([0,9,8,10,11])
vehicle_4 = route_time([0,13,12])
vehicle_5 = route_time([0,16,14,15])

print('V0 route time: ', vehicle_0)
print('V1 route time: ', vehicle_1)
print('V2 route time: ', vehicle_2)
print('V3 route time: ', vehicle_3)
print('V4 route time: ', vehicle_4)
print('V5 route time: ', vehicle_5)

print('\nTotal routing time: ', vehicle_0 + vehicle_1 + vehicle_2 + vehicle_3 + vehicle_4 + vehicle_5)

V0 route time:  87
V1 route time:  60
V2 route time:  69
V3 route time:  77
V4 route time:  70
V5 route time:  61

Total routing time:  424


## Optimization Setup

The Gurobipy package will be used to create an optimization model. The model, objective function, and constraints are established using the Gurobipy package.

In [11]:
import gurobipy as gp
from gurobipy import GRB

# initiate model
m = gp.Model('Delivery Routing')

Restricted license - for non-production use only - expires 2022-01-13


Next, we define the following three types of binary decision variables to formulate this problem. 

$$
\begin{array}{rl}
x_{ijk}& =\left\{\begin{array}{rl}1 & \text{if $V_k$ travels from $L_i$ to $L_j$}\\0&\text{otherwise}\end{array}\right.\\
y_{jk}& =\left\{\begin{array}{rl}1 & \text{if $V_k$ visits $L_j$}\\0&\text{otherwise}\end{array}\right.\\
z_{k}&=\left\{\begin{array}{rl}1 & \text{if $V_k$ is operating}\\0&\text{otherwise}\end{array}\right.\\
\end{array}
$$

In [12]:
# add decision variables

# binary variables for if a vehicle traveled from a departure to a destination
x = m.addVars(n, n, K, vtype=GRB.BINARY, name="x")

# binary variables for if a vehicle visited a location
y = m.addVars(n, K, vtype=GRB.BINARY, name="y")

# binary variable for if a vehicle is operating
z = m.addVars(K, vtype=GRB.BINARY, name="z")

In [13]:
# set objective function
m.setObjective(gp.quicksum(c[i,j]*x[i,j,k] 
                           for i in range(n)
                           for j in range(1, n)
                           for k in range(K)
                           ), 
               GRB.MINIMIZE
               )

In [14]:
# add constraint for if a vehicle has visited a location, that vehicle is active
m.addConstrs(y[i, k] <= z[k]
             for k in range(K)
             for i in range(1, n)
             )

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Upd

In [15]:
# add constraint that if a vehicle is active, it must have visited some location
m.addConstrs(y[0, k] >= z[k]
             for k in range(K)
             )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [16]:
# add constraint that no vehicle travels for more than 100 minutes
m.addConstrs(
    gp.quicksum(c[i, j] * x[i, j, k]
                for i in range(n)
                for j in range(1, n)
                ) <= T
             for k in range(K)
             )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [17]:
# add constraint that each location can only be visited by one vehicle
m.addConstrs(y.sum(i, '*') == 1
            for i in range(1, n))

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>}

In [18]:
# add constraints that if a vehicle visits a destination, the destination/vehicle must be active in the X matrix

m.addConstrs(x.sum('*', j, k) == y[j, k]
             for j in range(n)
             for k in range(K)
             )

m.addConstrs(x.sum(j,'*', k) == y[j, k]
             for j in range(n)
             for k in range(K)
             )

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,


In [19]:
# add constraint that lower indexed vehicles are utilized first before later indexed vehicles
m.addConstrs(y.sum('*', k-1) >= y.sum('*', k)
             for k in range(1, K)
             )

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [20]:
# add constraint that the departure and destination can't be the same location
m.addConstrs(x[i, i, k] == 0
             for i in range(n)
             for k in range(K)
             )

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,


## Avoiding Cycles

The next step is to ensure that a vehicle does not travel to a location multiple times (a 'cycle'). This code is provided by graduate coursework faculty.

In [21]:
from itertools import permutations
 
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = m.cbGetSolution(m._x)
        selected = gp.tuplelist((i,j) for i, j, k in m._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in range(K):
                m.cbLazy(gp.quicksum(m._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)
 
 
# Given a tuplelist of edges, find the shortest subtour not containing depot (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## Solve the Optimization Model

In [22]:
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 435 rows, 1842 columns and 5780 nonzeros
Model fingerprint: 0xe124cbdf
Variable types: 0 continuous, 1842 integer (1842 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [4e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 103 rows and 103 columns
Presolve time: 0.03s
Presolved: 332 rows, 1739 columns, 5468 nonzeros
Variable types: 0 continuous, 1739 integer (1739 binary)

Root relaxation: objective 1.743750e+02, 540 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  174.37500    0  210          -  174.37500      -     -    0s
   

In [23]:
# save the optimized solution
X = m.getAttr('X', x)

## Optimized Routing and Total Route Time

The code below provides the vehicles used, routes, routing time per vehicle, and total routing time for the optimized model. The code below was provided by graduate coursework faculty.

In [24]:
# Print optimal routes
for k in range(K):
    route = gp.tuplelist((i,j) for i in range(n) for j in range(n) if X[i,j,k] > 0.5)
    if route:
        i = 0
        print(f"Route for Vehicle {k}: \n{i}", end=' ')
        while True:
            i = route.select(i, '*')[0][1]
            if i == 0:
                break
            print(f" -> {i}", end='')
 
        print(f" \nTravel time: {sum(c[i,j]*X[i,j,k] for i in range(n) for j in range(1,n))} min", '\n')
 
print(f"Total Travel time: {round(sum(c[i,j]*X[i,j,k] for i in range(n) for j in range(1,n) for k in range(K)),0)} min")

Route for Vehicle 0: 
0  -> 16 -> 15 -> 2 -> 7 -> 14 -> 6 
Travel time: 72.0 min 

Route for Vehicle 1: 
0  -> 3 -> 13 -> 5 -> 10 -> 4 -> 11 
Travel time: 80.0 min 

Route for Vehicle 2: 
0  -> 9 -> 8 -> 1 -> 12 
Travel time: 48.0 min 

Total Travel time: 200.0 min


## Conclusion

The optimized routing uses 3 vehicles, which all depart from the headquarters. The total optimized routing time is 200 minutes.